In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import pandas
import joblib
import numpy as np

loaded_model = joblib.load('./water_test5')

spark = SparkSession\
  .builder\
  .master("local") \
  .appName("Kafka_stm")\
  .getOrCreate()

spark.sparkContext

/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator ColumnTransformer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own r

<SparkContext master=local appName=Kafka_stm>

In [ ]:
while True:


    df = spark\
         .read\
         .format("kafka")\
         .option("kafka.bootstrap.servers", "localhost:9092")\
         .option("subscribe", "waterQuality")\
         .load()

    # df.printSchema()

    # print(df.show())

    df1 = df.selectExpr("CAST(value AS STRING)")

    # print(df1.show())
    
    df_org = StructType()\
	.add("tbVal", StringType()) \
	.add("phVal", StringType()) \
	.add("liIndDivName", StringType()) \
	.add("clVal", StringType()) \
	.add("fcltyMngNm", StringType()) \
	.add("occrrncDt", StringType()) \
	.add("fcltyMngNo", StringType())

    df_void = StructType()\
        .add("clVal", StringType()) \
        .add("tbVal", StringType()) \
        .add("fcltyMngNo", StringType())

    df2 = df1.select(from_json(col("value"), df_void).alias("df"))
    df_org = df1.select(from_json(col("value"), df_org).alias("df"))
    
    # print(df2.show())

    df3 = df2.select("df.*")
    df4 = df3.selectExpr("CAST(clVal AS FLOAT)"\
        , "CAST(tbVal AS FLOAT)"\
        , "CAST(fcltyMngNo AS LONG)")

    # print(df4.show())
    # df4.printSchema()

    df5 = df4.toPandas()
    df_org = df_org.toPandas()
    df5.rename(columns = {'clVal':'cl', 'fcltyMngNo':'placeNo', 'phVal':'ph', 'tbVal':"tb"},inplace=True)

    data = df5.tail(1).to_numpy()
    data = data.astype('float64')

    # print(data)

    X_new = data

    # Use the model to predict tomorrow's rentals
    result = loaded_model.predict(X_new)

        
    date = time.strftime('%Y%m%d', time.localtime(time.time()))
    hour = time.strftime('%H', time.localtime(time.time()))
    prediction_val = float(np.round(result[0],4))
    out_data = {"date":date, "hour":hour, "predict_val":prediction_val}
    
    department1 = Row(Date=date, Hour=hour, Predict_val =prediction_val)

    departmentsWithEmployeesSeq1 = [department1]
    df_out = spark.createDataFrame(departmentsWithEmployeesSeq1)

    display(df_out)

    print(df_out.show())

    df_out = df_out.selectExpr("CAST(Date AS STRING)"\
            , "CAST(Hour AS STRING)"\
            , "CAST(Predict_val AS STRING)")

    df_ar = df_out.withColumn("value", array("Date", "Hour", "Predict_val"))

    print(df_ar.show())

    df_ar.selectExpr("CAST(value AS STRING)") \
          .write \
          .format("kafka") \
          .option("kafka.bootstrap.servers", "localhost:9092") \
          .option("topic", "water_predict") \
          .save()
    
    print("origin Data")
    print(df_org.tail(1))
    
    print('Prediction: {0} Ph'.format(np.round(result[0], 4)))
    #print(df1.show())
    #df1.printSchema()
    
    time.sleep(5)
    
    print("Generate New Prediction...")

DataFrame[Date: string, Hour: string, Predict_val: double]

+--------+----+-----------+
|    Date|Hour|Predict_val|
+--------+----+-----------+
|20210304|  13|     7.1103|
+--------+----+-----------+

None
+--------+----+-----------+--------------------+
|    Date|Hour|Predict_val|               value|
+--------+----+-----------+--------------------+
|20210304|  13|     7.1103|[20210304, 13, 7....|
+--------+----+-----------+--------------------+

None
origin Data
                                                  df
8  (0.0, 7.5469, 생활정수, 0.0, 고양정수장, None, 4128512319)
Prediction: 7.1103 Ph
Generate New Prediction...


DataFrame[Date: string, Hour: string, Predict_val: double]

+--------+----+-----------+
|    Date|Hour|Predict_val|
+--------+----+-----------+
|20210304|  13|     7.1103|
+--------+----+-----------+

None
+--------+----+-----------+--------------------+
|    Date|Hour|Predict_val|               value|
+--------+----+-----------+--------------------+
|20210304|  13|     7.1103|[20210304, 13, 7....|
+--------+----+-----------+--------------------+

None
origin Data
                                                  df
8  (0.0, 7.5469, 생활정수, 0.0, 고양정수장, None, 4128512319)
Prediction: 7.1103 Ph
Generate New Prediction...


DataFrame[Date: string, Hour: string, Predict_val: double]

...

+--------+----+-----------+
|    Date|Hour|Predict_val|
+--------+----+-----------+
|20210304|  14|     7.1103|
+--------+----+-----------+

None
+--------+----+-----------+--------------------+
|    Date|Hour|Predict_val|               value|
+--------+----+-----------+--------------------+
|20210304|  14|     7.1103|[20210304, 14, 7....|
+--------+----+-----------+--------------------+

None
origin Data
                                                  df
8  (0.0, 7.5469, 생활정수, 0.0, 고양정수장, None, 4128512319)
Prediction: 7.1103 Ph
Generate New Prediction...


DataFrame[Date: string, Hour: string, Predict_val: double]

+--------+----+-----------+
|    Date|Hour|Predict_val|
+--------+----+-----------+
|20210304|  14|     7.1103|
+--------+----+-----------+

None
+--------+----+-----------+--------------------+
|    Date|Hour|Predict_val|               value|
+--------+----+-----------+--------------------+
|20210304|  14|     7.1103|[20210304, 14, 7....|
+--------+----+-----------+--------------------+

None
origin Data
                                                  df
8  (0.0, 7.5469, 생활정수, 0.0, 고양정수장, None, 4128512319)
Prediction: 7.1103 Ph
Generate New Prediction...


DataFrame[Date: string, Hour: string, Predict_val: double]

+--------+----+-----------+
|    Date|Hour|Predict_val|
+--------+----+-----------+
|20210304|  14|     7.1103|
+--------+----+-----------+

None
+--------+----+-----------+--------------------+
|    Date|Hour|Predict_val|               value|
+--------+----+-----------+--------------------+
|20210304|  14|     7.1103|[20210304, 14, 7....|
+--------+----+-----------+--------------------+

None
origin Data
                                                  df
8  (0.0, 7.5469, 생활정수, 0.0, 고양정수장, None, 4128512319)
Prediction: 7.1103 Ph


In [ ]:

department1 = Row(Date=date, Hour=hour, Predict_val =prediction_val)

departmentsWithEmployeesSeq1 = [department1]
df_out = spark.createDataFrame(departmentsWithEmployeesSeq1)

display(df_out)

print(df_out.show())

df_out = df_out.selectExpr("CAST(Date AS STRING)"\
        , "CAST(Hour AS STRING)"\
        , "CAST(Predict_val AS STRING)")

df_ar = df_out.withColumn("value", array("Date", "Hour", "Predict_val"))

print(df_ar.show())

df_ar.selectExpr("CAST(value AS STRING)") \
      .write \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("topic", "water_predict") \
      .save()



# df_out = array(df_out.select("Date","Hour","Predict_val").collect())
# feature_columns = df_out.columns[0:] # here we omit the final column
# assembler = VectorAssembler(inputCols=feature_columns,outputCol="value")
# df_out = assembler.transform(df_out)

df_out.printSchema()